In [ ]:
#pip install pyzbar

In [ ]:
import cv2
from pyzbar.pyzbar import decode
import time

cam = cv2.VideoCapture(0)
cam.set(3, 640)  # Set width
cam.set(4, 480)  # Set height

camera = True
while camera:
    success, frame = cam.read()

    for i in decode(frame):
        print(i.type)
        print(i.data.decode('utf-8'))
        time.sleep(6)

    cv2.imshow("Our_QR_Code_Scanner", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit the loop
        break

cam.release()
cv2.destroyAllWindows()

In [ ]:
#pip install PyPDF2

In [ ]:
import PyPDF2
import re
import requests
from io import BytesIO

def extract_medicine_info(pdf_content):
    pdf_reader = PyPDF2.PdfReader(pdf_content)
    
    medicine_info = []
    
    for page_num in range(len(pdf_reader.pages)):
        text = pdf_reader.pages[page_num].extract_text()
        
        # Look for the headers of the columns we're interested in
        header_match = re.search(r'MEDICINE\s+NAME/DOSAGE.*?AMOUNT', text, re.IGNORECASE)
        if not header_match:
            continue

        header_start, header_end = header_match.span()
        
        # Extract the content after the headers
        content_after_headers = text[header_end:]
        
        # Debugging statement
        print("Content After Headers:", content_after_headers)
        
        # Updated regular expression to capture medicine names and amounts
        medicine_matches = re.findall(r'(\w+\s+\d+)', content_after_headers)
        
        # Debugging statement
        print("Matched Information:", medicine_matches)
        
        for match in medicine_matches:
            # Split the matched string into medicine name and amount
            medicine_name, amount = match.split()
            medicine_info.append({'Medicine Name': medicine_name, 'Amount': int(amount)})
    
    return medicine_info

# Replace 'https://me-qr.com/CVqqk6by' with the actual URL to your PDF
pdf_url = 'https://me-qr.com/CVqqk6by'
response = requests.get(pdf_url)
pdf_content = BytesIO(response.content)

# Display the extracted medicine information
medicine_info = extract_medicine_info(pdf_content)
for info in medicine_info:
    print(f"Medicine Name: {info['Medicine Name']}, Amount: {info['Amount']}")


In [ ]:
import PyPDF2
import re
import requests
from io import BytesIO

def extract_medicine_info(pdf_content):
    pdf_reader = PyPDF2.PdfReader(pdf_content)
    
    medicine_info = []
    
    for page_num in range(len(pdf_reader.pages)):
        text = pdf_reader.pages[page_num].extract_text()
        
        # Look for the headers of the columns we want to exclude
        exclude_headers = ['MEDICINE NAME/DOSAGE', 'AMOUNT']
        exclude_pattern = '|'.join(map(re.escape, exclude_headers))
        exclude_match = re.search(exclude_pattern, text, re.IGNORECASE)
        
        # Skip the page if any exclude header is found
        if exclude_match:
            continue

        # Updated regular expression to capture medicine names and amounts
        medicine_matches = re.findall(r'(\w+\s+\d+)', text)
        
        for match in medicine_matches:
            # Split the matched string into medicine name and amount
            medicine_name, amount = match.split()
            medicine_info.append({'Medicine Name': medicine_name, 'Amount': int(amount)})
    
    return medicine_info

# Replace 'https://me-qr.com/CVqqk6by' with the actual URL to your PDF
pdf_url = 'https://me-qr.com/CVqqk6by'
response = requests.get(pdf_url)
pdf_content = BytesIO(response.content)

# Display the extracted medicine information
medicine_info = extract_medicine_info(pdf_content)
for info in medicine_info:
    print(f"Medicine Name: {info['Medicine Name']}, Amount: {info['Amount']}")


In [ ]:
#pip install tabula-py

In [ ]:
import fitz  # PyMuPDF
import re
import requests
from io import BytesIO

def extract_table_from_pdf(pdf_content):
    doc = fitz.open(stream=pdf_content, filetype="pdf")
    
    # Iterate through pages and extract text
    table_data = []
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text = page.get_text()
        
        # Modify the regular expression based on the structure of your table
        table_matches = re.findall(r'(\w+\s+\d+)', text)
        
        for match in table_matches:
            # Split the matched string into columns (modify as needed)
            columns = match.split()
            table_data.append(columns)
    
    doc.close()
    return table_data

# Example usage:
# Replace 'https://example.com/your-pdf.pdf' with the actual URL to your PDF
pdf_url = 'https://scnv.in/H0Tt?qr=1'
response = requests.get(pdf_url)
pdf_content = BytesIO(response.content)

# Display the extracted table
table_data = extract_table_from_pdf(pdf_content)
for row in table_data:
    print(row)
